# Teaching LangChain Function Calling for Beginners

Welcome to this beginner-friendly Colab notebook! We'll learn about **LangChain Function Calling** (also known as Tool Calling). This feature lets Large Language Models (LLMs) decide when to call external "tools" (like APIs) to get real-world data, instead of just generating text from their training.

By the end, you'll build a simple app that asks an LLM for the current weather in Hong Kong using a real weather API.

**What you'll need:**
- A free Google Colab account.
- An API key for the Azure OpenAI endpoint (provided in the docs: `https://aai02.eduhk.hk/openai/deployments/gpt-4o-mini/chat/completions`). If you don't have one, ask your instructor or use a placeholder.

Let's get started!

## Step 1: Install Required Packages

Run this cell to install LangChain, the OpenAI integration, and Requests (for API calls).

In [1]:
!pip install -q langchain langchain_openai requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00


## Step 2: Import Libraries and Set Up Your API Key

We'll import the necessary modules and set up the Azure OpenAI model. Replace `'your-api-key-here'` with your actual API key.

In [2]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
import requests
from datetime import datetime
import json

from google.colab import userdata


# Set your Azure OpenAI API key (keep it secret! In Colab, you can use os.environ for security)
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('eduhkkey')

# Set up the Azure OpenAI model (using gpt-4o-mini as per docs)
llm = AzureChatOpenAI(
    azure_endpoint="https://aai02.eduhk.hk/openai/deployments/gpt-4o-mini/chat/completions?Hello=",
    api_version="2024-02-15-preview",  # Use a recent version
    deployment_name="gpt-4o-mini",
    temperature=0,  # Low temperature for consistent tool calling
    streaming=False,  # Non-streaming for simplicity
)

# The actual endpoint used internally
print(f"Base URL: {llm.client._client._base_url}")
print(f"API Version: {llm.openai_api_version}")
print(f"Deployment: {llm.deployment_name}")
print(os.environ["AZURE_OPENAI_API_KEY"])

Base URL: https://aai02.eduhk.hk/openai/deployments/gpt-4o-mini/chat/completions?Hello=/openai/deployments/gpt-4o-mini/
API Version: 2024-02-15-preview
Deployment: gpt-4o-mini
f0Ml4DrKVyeXt19VrdGmmzrqJffWfABGLxTiaeaNkeAZxVX3prznwKxpDoS2H6UXzGLPVdxO


**Quick Explanation:**
- `AzureChatOpenAI` connects to the Azure endpoint you provided.
- `temperature=0` makes the model more deterministic (good for beginners).
- We're using non-streaming mode to get full responses at once.

## Step 3: Understand Function Calling

**What is Function Calling?**
- LLMs like GPT can't access the internet or real-time data directly.
- Function Calling lets the LLM "call" a predefined function (tool) with parameters.
- Example: User asks "What's the weather in Hong Kong?" → LLM calls a `get_current_weather` tool → Tool fetches data from an API → LLM summarizes it.

In LangChain:
- Define tools with the `@tool` decorator (includes name, description, and parameters).
- "Bind" the tool to the LLM so it knows when/how to use it.
- Invoke the LLM with a user query, and it handles the rest!

Our tool will use the **Hong Kong Observatory (HKO) Open Data Weather API** to get real-time weather (e.g., temperature, humidity).

## Step 4: Define the "Get Current Weather" Tool

We'll create a tool that calls the HKO API for current weather reports (`dataType=rhrread`). It's Hong Kong-specific, so no location parameter needed for simplicity.

The tool fetches JSON data and returns a formatted string.

In [3]:
@tool
def get_current_weather(place: str) -> str:
    """
    Get the current weather in a given place (e.g., 'Hong Kong').

    This tool uses the Hong Kong Observatory API to fetch real-time data like temperature and humidity.
    For now, it only works for places in Hong Kong.

    Args:
        place (str): The place name (e.g., "Hong Kong").

    Returns:
        str: A summary of the current weather.
    """
    if "hong kong" not in place.lower():
        return "Sorry, this tool only supports weather for Hong Kong locations."

    # HKO API endpoint for current weather report
    url = "https://data.weather.gov.hk/weatherAPI/opendata/weather.php"
    params = {
        "dataType": "rhrread",
        "lang": "en"  # English
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Raise error for bad status
        data = response.json()

        # Extract key info (corrected paths based on actual API structure)
        update_time = data.get("updateTime", "Unknown")
        temperature_data = data.get("temperature", {}).get("data", [])
        humidity_data = data.get("humidity", {}).get("data", [])
        warnings = data.get("warningMessage", []) or data.get("specialWxTips", [])

        # Get first station's temp and humidity (e.g., King's Park or HKO)
        temp = temperature_data[0]["value"] if temperature_data else "N/A"
        hum = humidity_data[0]["value"] if humidity_data else "N/A"

        summary = f"Current weather in Hong Kong (reported at {update_time}):\n"
        summary += f"- Temperature: {temp}°C\n"
        summary += f"- Humidity: {hum}%\n"

        if warnings:
            summary += f"- Warnings/Tips: {'; '.join(warnings)}\n"

        return summary

    except Exception as e:
        return f"Error fetching weather: {str(e)}"

**Quick Explanation:**
- `@tool` turns the function into a LangChain tool. The docstring describes it for the LLM.
- We use `requests.get` to call the HKO API (base URL from docs).
- Parse the JSON response (structure from HKO docs: nested `data` with `temperature`, `humidity`, etc.).
- Return a human-readable string. In a real app, you'd handle more fields!

Test the tool standalone:

In [4]:
# Quick test of the tool
print(get_current_weather.invoke({"place": "Hong Kong"}))

Current weather in Hong Kong (reported at 2025-09-24T20:21:00+08:00):
- Temperature: 26°C
- Humidity: 93%
- Warnings/Tips: The Tropical Cyclone Signal No. 3 was issued at 8:20 p.m.; The Landslip Warning has been issued.; The Special Announcement of Flooding in Northern New Territories has been issued.



## Step 5: Bind the Tool to the LLM

Now, "bind" the tool to our LLM. This tells the model it can use `get_current_weather` when needed.

In [5]:
# Bind the tool to the LLM
llm_with_tools = llm.bind_tools([get_current_weather])

# Print the tool schema directly (what the LLM sees)
print(get_current_weather.get_input_schema())

<class 'langchain_core.utils.pydantic.get_current_weather'>


**What happens under the hood?**
- The LLM gets the tool's schema (name, description, parameters).
- When you query, the LLM decides: "Do I need this tool?" If yes, it outputs a "tool call" with args.
- LangChain executes the tool and feeds the result back to the LLM for a final response.

## Step 6: Create a Simple Chain and Run Examples

We'll use `create_tool_calling_agent` for a basic agent that handles tool calls automatically. (this is easier than manual parsing.)

In [6]:
# Fixed Step-by-Step Forcing: JSON-Only Tool Decision + Clean Summary
from langchain_core.messages import HumanMessage, SystemMessage
import json

def force_tool_via_json(user_input: str):
    # Phase 1: Prompt for JSON-only tool decision (strict format)
    json_prompt = SystemMessage(content="""You are a tool-calling assistant. Analyze the user query and respond
    **ONLY** with valid JSON in this exact format:
    {
      "use_tool": true/false,  // true if query is about CURRENT weather in Hong Kong
      "tool_name": "get_current_weather" or null,
      "args": {"place": "string"} or null  // e.g., {"place": "Hong Kong"}
    }
    Do NOT output any other text, explanations, or markdown. If not current weather, set use_tool=false.""")

    messages = [json_prompt, HumanMessage(content=user_input)]

    try:
        json_response = llm_with_tools.invoke(messages)  # Use bound LLM for schema awareness
        json_str = json_response.content.strip()  # Extract content
        print("Raw JSON Output:", json_str)  # Debug: See what it outputs

        # Phase 2: Parse and execute
        tool_plan = json.loads(json_str)
        if tool_plan.get("use_tool") and tool_plan.get("tool_name") == "get_current_weather":
            args = tool_plan.get("args", {})
            tool_result = get_current_weather.invoke(args)
            print("Tool Executed! Result:", tool_result)

            # Phase 3: NEW fresh messages for summarization (no JSON prompt!)
            summary_system = SystemMessage(content="""You are a helpful weather assistant.
            Summarize the provided tool result in a natural, friendly response to the user's query.
            Include key details like temperature, humidity, and warnings. Keep it concise and engaging.""")

            # Fresh chain: User query + tool result only
            summary_messages = [
                summary_system,
                HumanMessage(content=user_input),
                HumanMessage(content=f"Tool result: {tool_result}")
            ]
            final_response = llm.invoke(summary_messages)  # Use unbound LLM for free-form text
            return final_response.content
        else:
            return "No tool needed for this query."
    except json.JSONDecodeError:
        return "Error: LLM didn't output valid JSON. Try rephrasing."
    except Exception as e:
        return f"Error: {e}"

# Test it
user_query = "What's the current weather in Hong Kong?"
result = force_tool_via_json(user_query)
print("Final Summary:", result)

Raw JSON Output: {
  "use_tool": true,
  "tool_name": "get_current_weather",
  "args": {"place": "Hong Kong"}
}
Tool Executed! Result: Current weather in Hong Kong (reported at 2025-09-24T20:21:00+08:00):
- Temperature: 26°C
- Humidity: 93%
- Warnings/Tips: The Tropical Cyclone Signal No. 3 was issued at 8:20 p.m.; The Landslip Warning has been issued.; The Special Announcement of Flooding in Northern New Territories has been issued.

Final Summary: Hello! Right now, it's a warm 26°C in Hong Kong with quite high humidity at 93%. Please be aware that there are several warnings in effect: a Tropical Cyclone Signal No. 3 was issued, a Landslip Warning is active, and there's also a Special Announcement regarding flooding in the Northern New Territories. Stay safe and take care!


## Step 7: What's Next?

Congrats! You've built your first LangChain Function Calling app. How to apply this to your project?

**Resources:**
- [LangChain Docs: Tools](https://python.langchain.com/docs/modules/agents/tools/)
- [HKO API Full Docs](https://www.hko.gov.hk/en/weatherAPI/doc/files/HKO_Open_Data_API_Documentation.pdf)
- Questions? Ask in the comments!

Happy coding! 🌤️